https://www.kaggle.com/tsiaras/uk-road-safety-accidents-and-vehicles

In [1]:
#Import modules
import numpy as np
import holidays
import pandas as pd
import datetime
import sklearn
import scipy
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import math
from sklearn.model_selection import cross_val_score
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
#import files

df1 = pd.read_csv('Accident_Information.csv')
df2 = pd.read_csv('Vehicle_Information.csv')

In [3]:
#check columns
print("Accident's Columns:\n",df1.columns, "\n")

print("Vehicle's Columns:\n",df2.columns)

Accident's Columns:
 Index(['Accident_Index', '1st_Road_Class', '1st_Road_Number', '2nd_Road_Class',
       '2nd_Road_Number', 'Accident_Severity', 'Carriageway_Hazards', 'Date',
       'Day_of_Week', 'Did_Police_Officer_Attend_Scene_of_Accident',
       'Junction_Control', 'Junction_Detail', 'Latitude', 'Light_Conditions',
       'Local_Authority_(District)', 'Local_Authority_(Highway)',
       'Location_Easting_OSGR', 'Location_Northing_OSGR', 'Longitude',
       'LSOA_of_Accident_Location', 'Number_of_Casualties',
       'Number_of_Vehicles', 'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Police_Force',
       'Road_Surface_Conditions', 'Road_Type', 'Special_Conditions_at_Site',
       'Speed_limit', 'Time', 'Urban_or_Rural_Area', 'Weather_Conditions',
       'Year', 'InScotland'],
      dtype='object') 

Vehicle's Columns:
 Index(['Accident_Index', 'Age_Band_of_Driver', 'Age_of_Vehicle',
       'Driver_Home_Area_Type', 'Driver_IMD_Decile', '

In [4]:
#merge dataframes
df3 = df1.merge(df2, on='Accident_Index')

In [5]:
#check
df3.head()

,Accident_Index,1st_Road_Class,1st_Road_Number,2nd_Road_Class,2nd_Road_Number,Accident_Severity,Carriageway_Hazards,Date,Day_of_Week,Did_Police_Officer_Attend_Scene_of_Accident,...,Skidding_and_Overturning,Towing_and_Articulation,Vehicle_Leaving_Carriageway,Vehicle_Location.Restricted_Lane,Vehicle_Manoeuvre,Vehicle_Reference,Vehicle_Type,Was_Vehicle_Left_Hand_Drive,X1st_Point_of_Impact,Year_y
0,200501BS00002,B,450.0,C,0.0,Slight,None,2005-01-05,Wednesday,1.0,...,None,No tow/articulation,Did not leave carriageway,0.0,Slowing or stopping,1,Bus or coach (17 or more pass seats),No,Nearside,2005
1,200501BS00003,C,0.0,NaN,0.0,Slight,None,2005-01-06,Thursday,1.0,...,None,No tow/articulation,Did not leave carriageway,0.0,Going ahead right-hand bend,1,Bus or coach (17 or more pass seats),No,Nearside,2005
2,200501BS00004,A,3220.0,NaN,0.0,Slight,None,2005-01-07,Friday,1.0,...,None,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,1,Car,No,Front,2005
3,200501BS00005,Unclassified,0.0,NaN,0.0,Slight,None,2005-01-10,Monday,1.0,...,Skidded,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,1,Motorcycle 125cc and under,No,Front,2005
4,200501BS00006,Unclassified,0.0,NaN,0.0,Slight,None,2005-01-11,Tuesday,1.0,...,None,No tow/articulation,Did not leave carriageway,0.0,Moving off,1,Car,No,Did not impact,2005


In [6]:
#check columns and datatypes
print("Column's Datatypes:\n\n",df3.dtypes)

Column's Datatypes:

 Accident_Index                                  object
1st_Road_Class                                  object
1st_Road_Number                                float64
2nd_Road_Class                                  object
2nd_Road_Number                                float64
Accident_Severity                               object
Carriageway_Hazards                             object
Date                                            object
Day_of_Week                                     object
Did_Police_Officer_Attend_Scene_of_Accident    float64
Junction_Control                                object
Junction_Detail                                 object
Latitude                                       float64
Light_Conditions                                object
Local_Authority_(District)                      object
Local_Authority_(Highway)                       object
Location_Easting_OSGR                          float64
Location_Northing_OSGR                     

In [7]:
#some column cleanups

#change " Accident_Index" to "Accident_Index"
df3 = df3.rename(columns={' Accident_Index':'Accident_Index'})

#delete "Year_y" and change "Year_x" to "Year" (duplicates)
df3 = df3.drop(['Year_y'], axis=1)

df3 = df3.rename(columns={'Year_x':'Year'})

In [8]:
#check corr b/t Location_Easting_OSGR & Location_Northing_OSGR AND Longitude and Latitude

print(df3['Location_Easting_OSGR'].corr(df3['Longitude']))


print(df3['Location_Northing_OSGR'].corr(df3['Latitude']))

0.9993670195112463
0.9999728865576533


In [9]:
#drop Location_Easting_OSGR & Location_Northing_OSGR
#because they are the similar to Latitude and Longitude 

df3 = df3.drop(['Location_Easting_OSGR', 'Location_Northing_OSGR'], axis=1)

In [10]:
#UNCOMMENT THIS COLUMN AND THE NEXT AFTER SOME ANALYSIS MAYBE
# #check % of null values
# df3.isnull().sum().sort_values(ascending=False)/df3.shape[0]*100

In [11]:
#standardize all column names to lowercase, and remove some characters 
#for ease of use in querying
df3.columns = map(str.lower, df3.columns)
df3.columns = df3.columns.str.replace('.','')
df3.columns = df3.columns.str.replace('(','')
df3.columns = df3.columns.str.replace(')','')

In [12]:
#import UK holidays
year = range(2005,2018)
uk_holidays = holidays.UnitedKingdom(years=year)

In [13]:
#create UK holiday dataframes
ukh  = pd.DataFrame(list(uk_holidays.items()))

In [14]:
#rename columns
ukh = ukh.rename(columns={0:"date", 1:"holiday"})

In [15]:
#combine holiday with traffic dataframe
df = pd.concat([df3,ukh])

In [17]:
df_num=df.select_dtypes(exclude=['object'])

df_Zscore=df_num[(np.abs(stats.zscore(df_num))<3).all(axis=1)]

In [ ]:
#Check Missing Data
df.isnull().sum().sort_values(ascending=False)/df.shape[0]*100

In [ ]:
#adding holidays created a lot of missing values that needed to be deleted or fixed

#filled null holiday spots with "Non-Holiday"
df['holiday'].fillna("Non-Holiday", inplace=True)

#dropped null 'accident_index' rows that came about after adding holidays to the dataframe
df = df.dropna(subset=['accident_index'])

In [ ]:
#check values of vehicle age before handling nulls

print("Vehicle Age Values")
print("Average Vehicle Age: ", df['age_of_vehicle'].mean())
print("Minimum Vehicle Age: ", df['age_of_vehicle'].min())
print("Median of Vehicle Age", df['age_of_vehicle'].median())
print("Maximum Vehicle Age: ", df['age_of_vehicle'].max())
print("Null values: ", pd.isnull(df['age_of_vehicle']).sum())

In [ ]:
#A car being 111 years old IS possible so I'll leave it
#Next I'm going to replace the null values of the vehicle ages with the mean & mediam of 7

#age_of_vehicle
df['age_of_vehicle'].fillna(df['age_of_vehicle'].mean(),inplace=True)


In [19]:
df['propulsion_code'].value_counts()/df.shape[0]*100

Petrol                 63.474802
Heavy oil              35.928093
Hybrid electric         0.445352
Gas/Bi-fuel             0.077043
Petrol/Gas (LPG)        0.030263
Electric                0.028855
Gas                     0.007578
Electric diesel         0.006946
New fuel technology     0.000534
Gas Diesel              0.000340
Steam                   0.000146
Fuel cells              0.000049
Name: propulsion_code, dtype: float64

In [18]:
#decided to fill the na for propulsion code with petrol. (The majority of cars are petrol/gas)
df['propulsion_code'].fillna('Petrol', inplace=True)

In [ ]:
#Change Gas to Petrol
df['propulsion_code'] = df['propulsion_code'].replace(to_replace="Gas", value="Petrol")
df['propulsion_code'] = df['propulsion_code'].replace(to_replace="Gas/Bi-fuel", value="Bio-fuel")
df['propulsion_code'] = df['propulsion_code'].replace(to_replace="Petrol/Gas (LPG)", value="LPG Petrol")
df['propulsion_code'] = df['propulsion_code'].replace(to_replace="Gas Diesel", value="Diesel")

In [ ]:
df['propulsion_code'].value_counts()/df.shape[0]*100

In [20]:
#Check Missing Data
df.isnull().sum().sort_values(ascending=False)/df.shape[0]*100

holiday                                        99.990770
2nd_road_class                                 40.227718
driver_imd_decile                              33.492697
age_of_vehicle                                 16.426325
model                                          14.551020
engine_capacity_cc                             12.168816
lsoa_of_accident_location                       6.771453
make                                            5.393719
2nd_road_number                                 0.928642
pedestrian_crossing-physical_facilities         0.075780
vehicle_locationrestricted_lane                 0.063830
pedestrian_crossing-human_control               0.040999
time                                            0.016322
longitude                                       0.015302
latitude                                        0.015253
did_police_officer_attend_scene_of_accident     0.014767
speed_limit                                     0.012387
inscotland                     

In [21]:
df['2nd_road_class'].value_counts()/df.shape[0]*100

Unclassified    40.097824
A               10.119849
C                4.674103
B                3.999712
Motorway         0.796270
A(M)             0.084524
Name: 2nd_road_class, dtype: float64

In [ ]:
#the majority of 2nd road class is either unclassified (40.1%) or null (40.2%)
#dropping column

df = df.drop(['2nd_road_class'], axis=1)

In [ ]:
df['driver_imd_decile'].value_counts()/df.shape[0]*100

In [ ]:
print("Median", df['driver_imd_decile'].median())
print("Mean",df['driver_imd_decile'].mean())

In [ ]:
#both are around 5 so I'm going to fill the null values of driver_imd_decile with 5 
#and I'll do it through the median.
df['driver_imd_decile'].fillna(df['driver_imd_decile'].median(),inplace=True)

In [ ]:
df['driver_imd_decile'].value_counts()/df.shape[0]*100

In [ ]:
print("Median", df['driver_imd_decile'].median())
print("Mean",df['driver_imd_decile'].mean())

#not much change in mean and media after change

In [ ]:
df['model'].value_counts()/df.shape[0]*100

In [ ]:
#decided to fill the na for model with "model unknown"
df['model'].fillna("Model Unknown", inplace=True)

In [ ]:
df['model'].value_counts()/df.shape[0]*100

In [ ]:
df['make'].value_counts()/df.shape[0]*100

In [22]:
#check values of vehicle age before handling nulls

print("engine_capacity_cc Values")
print("Average engine_capacity_cc: ", df['engine_capacity_cc'].mean())
print("Minimum engine_capacity_cc: ", df['engine_capacity_cc'].min())
print("Median of engine_capacity_cc", df['engine_capacity_cc'].median())
print("Maximum engine_capacity_cc: ", df['engine_capacity_cc'].max())
print("Null values of engine_capacity_cc: ", pd.isnull(df['engine_capacity_cc']).sum())

engine_capacity_cc Values
Average engine_capacity_cc:  2028.2906977580221
Minimum engine_capacity_cc:  1.0
Median of engine_capacity_cc 1598.0
Maximum engine_capacity_cc:  96000.0
Null values of engine_capacity_cc:  250507


In [ ]:
#Check % Missing Data again
df.isnull().sum().sort_values(ascending=False)/df.shape[0]*100

In [ ]:
#pickling here bc of errors
df.to_pickle('./df.pkl')
